# Capture outputs

This notebook will demonstrate how to capture still frames or videos from pythreejs using [ipywebrtc](https://ipywebrtc.readthedocs.io/en/latest/).

## Setup an example renderer

In [ ]:
from pythreejs import *
import ipywebrtc
from ipywidgets import Output, VBox

In [ ]:
view_width = 600
view_height = 400

sphere = Mesh(
    SphereBufferGeometry(1, 32, 16),
    MeshStandardMaterial(color='red')
)

cube = Mesh(
    BoxBufferGeometry(1, 1, 1),
    MeshPhysicalMaterial(color='green'),
    position=[2, 0, 4]
)

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[sphere, cube, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

In [ ]:
renderer

## Capture renderer output to stream

In [ ]:
stream = ipywebrtc.WidgetStream(widget=renderer, max_fps=30)

If you want, you can preview the content of the stream with a video-viewer. This should simply mirror what you see in the renderer.

In [ ]:
stream

## Capturing images

To capture images from the stream, use the `ImageRecorder` widget from `ipywebrtc`.

In [ ]:
recorder = ipywebrtc.ImageRecorder(filename='snapshot', format='png', stream=stream)

There are two ways to capture images from the stream:
1. Manually from the browser by using the widget view of the recorder.
2. Programmatically using the .save()/download() method on the recorder.

### Using the view

In [ ]:
recorder

Here,clicking the "Snapshot" button will capture a new frame and sync it back to the kernel side. Clicking "Download" will download the current snapshot on the *client side*. When taking a snapshot, the image will also be synced to the *kernel side*. If the image has changed, any observers of the value trait of the image will trigger (i.e. `recorder.image.observe(callback, 'value')`):

In [ ]:
out = Output()  # To capture print output

@out.capture()
def on_capture(change):
    print('Captured image changed!')
recorder.image.observe(on_capture, 'value')
out

### Using kernel API:

To request a snapshot from the kernel, set the `recording` attribute of the recorder to `True`. This will update the `image` attribute asynchronously. The easiest way to save this to the kernel side is to also set the `filename` attribute, and set `autosave` to `True`. This will cause the image to be saved as soon as it is available. This is equivalend to observing the image widget's `value` trait, and calling the `save()` method when the image changes.

In [ ]:
recorder.autosave = True
recorder.recording = True

You can also trigger a client-side download from the kernel by calling the `download()` method on the recorder:

In [ ]:
recorder.download()

## Capturing video

To capture a video from the stream, use the `VideoRecorder` from `ipywebrtc`.

In [ ]:
video_recorder = ipywebrtc.VideoRecorder(stream=stream, filename='video', codecs='vp8')

In [ ]:
video_recorder

Here, clicking the "Record" button will start capturing the video. Once you click the "Stop" button (appears after clicking "Record"), the video will be displayed in the view, and it will be synced to the kernel. If the video has changed, any observers of the value trait of the video will trigger, similarly to that of the `ImageRecorder`. Clicking "Download" will download the current video on the client side.

The kernel side API for the `VideoRecorder` is similar to that of the `ImageRecorder`, but you will also have to tell it when to stop:

In [ ]:
video_recorder.autosave = True
video_recorder.recording = True
# After executing this, try to interact with the renderer above before executing the next cell

In [ ]:
video_recorder.recording = False
video_recorder.download()